# 使用低级别 TensorFlow API (TensorFlow Core) 开始编程

## 目录
- 使用 tf.Graph 和 tf.Session 管理 TensorFlow 程序

- 使用 tf.Session 运行 TensorFlow 操作。
- 在此低级别环境中使用高级别组件（数据集、层和 feature_columns）。
- 构建自己的训练循环，而不是使用 Estimator 提供的训练循环。<br>

最好还是尽可能使用更高阶的 API 构建模型。
以下是 TensorFlow 低级API 为何很重要的原因：

- 如果您能够直接使用低阶 TensorFlow 操作，实验和调试都会更直接。
- 在使用更高阶的 API 时，能够理解其内部工作原理。

## 引入需要的包

In [31]:
import numpy as np
import tensorflow as tf

## Tensor Values （张量）
TensorFlow 中的核心数据单位是张量(**tensor**)。一个张量由一组形成阵列（**任意维数**）的原始值组成。张量的阶(**rank**)是它的维数，而它的形状(**shape**)是一个整数元组，指定了阵列每个维度的长度。<br>
TensorFlow 使用 numpy 阵列来表示张量值。<br>

以下是张量值的一些示例：

In [32]:
# 一个阶为0的tensor, shape是[]的标量
3. 
# 一个阶为1的tensor, shape是[3]的向量
[1., 2., 3.] 
# 一个阶为2的tensor, shape是[2， 3]的矩阵
[[1., 2., 3.], [4., 5., 6.]] 
# 一个阶为3的tensor, shape是[2, 1, 3]的矩阵
[[[1., 2., 3.]], [[7., 8., 9.]]]

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

***请前往03_1_tensor, 03_2_variable***, 获得详细的tensor介绍

## Tensorflow Core
可以将 TensorFlow Core 程序看作由两个互相独立的部分组成：

- 构建计算图 (tf.Graph)。
- 运行计算图（使用 tf.Session）。

### 计算图
***一般而言，使用TensorFlow程序的流程是先创建一个图，然后在session中启动它。***

计算图是排列成一个图的一系列 TensorFlow 指令。图由两种类型的对象组成。

- 操作（简称“op”）：图的节点。操作描述了消耗和生成张量的计算。
- 张量：图的边。它们代表将流经图的值。大多数 TensorFlow 函数会返回 tf.Tensors。
我们来构建一个简单的计算图。<br>
最基本的指令是一个常量。构建指令的 Python 函数将一个张量值作为输入值。<br>
生成的指令不需要输入值。它在运行时输出的是被传递给构造函数的值。<br>
我们可以创建如下所示的两个浮点数常量 a 和 b：

In [33]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const_10:0", shape=(), dtype=float32)
Tensor("Const_11:0", shape=(), dtype=float32)
Tensor("add_8:0", shape=(), dtype=float32)


**注意**，打印张量并不会如预期的那样输出值 3.0、4.0 和 7.0。<br>
上述语句只会构建计算图。这些 tf.Tensor 对象仅代表将要运行的操作的结果。

图中的每个指令都拥有唯一的名称。这个名称不同于使用 Python 分配给相应对象的名称。张量是根据生成它们的指令命名的，后面跟着输出索引，如 "add:0"。

## 会话 (Session)
要评估张量，需要实例化一个 tf.Session 对象）。会话会封装 TensorFlow 运行时的状态，并运行 TensorFlow 操作。<br>
如果说 tf.Graph 像一个 .py 文件，那么 tf.Session 就像一个 python 可执行对象。

下面的代码会创建一个 tf.Session 对象，然后调用其 run 方法来评估我们在上文中创建的 total 张量：

In [34]:
sess = tf.Session()
print(sess.run(total))

7.0


当使用 Session.run 请求输出节点时，TensorFlow 会回溯整个图，<br>
并流经提供了所请求的输出节点对应的输入值的所有节点。因此此指令会打印预期的值 7.0：<br>
可以将多个张量传递给 tf.Session.run。run 方法以透明方式处理元组或字典的任何组合，如下例所示：

In [37]:
print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


在调用 tf.Session.run 期间，任何 tf.Tensor 都只有单个值。<br>
例如，以下代码调用 tf.random_uniform 来生成一个 tf.Tensor，后者会生成随机的三元素矢量（值位于 [0,1) 区间内）。<br>
每次调用 run 时，结果都会显示不同的随机值，但在单个 run 期间（out1 和 out2 接收到相同的随机输入值），结果显示的值是一致的：

In [40]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.98193777 0.8016151  0.04787409]
[0.67761266 0.88284373 0.74878335]
(array([1.588478 , 1.4116062, 1.8086885], dtype=float32), array([2.588478 , 2.4116063, 2.8086886], dtype=float32))


### Feeding
目前的图只能使用常量值，因此它总是生成一个常量结果。<br>
图可以参数化以便接受外部输入，也称为占位符。占位符表示在稍后提供值，它就像函数参数。

In [41]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

前面三行有点像函数。我们定义了这个函数的两个输入参数（x 和 y），然后对它们指令。<br>
我们可以使用 run 方法的 feed_dict 参数为占位符提供具体的值，从而评估这个具有多个输入的图：

In [42]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


## 层 （Layer）
可训练的模型必须修改图中的值，以便在输入相同值的情况下获得新的输出值。将可训练参数添加到图中的首选方法是层。<br>

层将变量和作用于它们的操作打包在一起。例如，全连接层会对每个输出对应的所有输入执行加权和，并应用激活函数（可选）。连接权重和偏差由层对象管理。
### 创建层

In [43]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

层会检查其输入数据，以确定其内部变量的大小。因此，我们必须在这里设置 x 占位符的形状，以便层构建正确大小的权重矩阵。

我们现在已经定义了输出值 y 的计算，在我们运行计算之前，还需要处理一个细节。
### 初始化层

In [44]:
init = tf.global_variables_initializer()
sess.run(init)

global_variables_initializer 仅会初始化创建初始化程序时图中就存在的变量。因此应该在构建图表的最后一步添加初始化程序。
### 执行层
我们现在已经完成了层的初始化，可以像处理任何其他张量一样评估 linear_model 的输出张量了。例如，下面的代码：

In [45]:
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

[[ 4.778865]
 [10.312775]]


## 训练
已经了解 TensorFlow 核心部分的基础知识了，我们来手动训练一个小型回归模型吧。
### 定义数据
我们首先来定义一些输入值 `x`，以及每个输入值的预期输出值 `y_true`：

In [46]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

### 定义模型
接下来，建立一个简单的线性模型，其输出值只有 1 个：

In [47]:
linear_model = tf.layers.Dense(units=1)

# 进行预测、还没有训练，所以效果不是很好
y_pred = linear_model(x)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[0.5243298]
 [1.0486596]
 [1.5729893]
 [2.0973191]]


### 损失
要优化模型，首先需要定义损失。我们将使用均方误差，这是回归问题的标准损失。

虽然可以使用较低级别的数学运算手动定义，但 tf.losses 模块提供了一系列常用的损失函数。可以使用它来计算均方误差，具体操作如下所示：

In [48]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

10.805211


### 训练
TensorFlow 提供了执行标准优化算法的优化器。这些优化器被实现为 tf.train.Optimizer 的子类。它们会逐渐改变每个变量，以便将损失最小化。最简单的优化算法是梯度下降法，由 tf.train.GradientDescentOptimizer 实现。它会根据损失相对于变量的导数大小来修改各个变量。例如：

In [49]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

该代码构建了优化所需的所有图组件，并返回一个训练指令。该训练指令在运行时会更新图中的变量。您可以按以下方式运行该指令：

In [50]:
for i in range(10):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

10.805211
7.590361
5.35909
3.8103056
2.7350888
1.9884733
1.4698704
1.1094842
0.85888433
0.684466


## 实践
请将上面的程序写成一个完整的程序